In [ ]:
import datetime
import os, math
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import persistence_diagram
import tensorflow as tf
import layers
import numpy as np
import utils

man_dim = 2  # Dimension of the manifold
K = 28  # number of projection bases
num_of_hom = 2  # number of homology classes; hardcoded for now (we know its 3 for images);
# TODO code function to find it

batch_size = 512
epochs = 3
save_every = 1


def pad_diagrams(dgms):
    '''
        Adds zeros points to the diagrams to make them of equal size; tensorflow cant handle inputs with varied size
    '''
    # Get the highest number of points in a PD
    max_num_of_pnts = 0
    for ind in dgms.keys():
        num_of_dgms = 0
        for filtration in dgms[ind].keys():
            for par_ind in dgms[ind][filtration].keys():
                dgm = dgms[ind][filtration][par_ind]
                max_num_of_pnts = max(max_num_of_pnts, dgm.shape[0])
                num_of_dgms += 1

    # Pad
    max_num_of_pnts = 77
    N = len(dgms.keys())
    out = np.zeros([N, num_of_dgms, num_of_hom, max_num_of_pnts, man_dim], dtype=np.float32)
    out[:,:,:,2] = -1 
    for ind in dgms.keys():
        cnt = 0
        for filtration in dgms[ind].keys():
            for par_ind in dgms[ind][filtration].keys():
                dgm = dgms[ind][filtration][par_ind]
                for p_ind in range(dgm.shape[0]):
                    hom = int(dgm[p_ind,2])
                    out[ind, cnt, hom, p_ind, :] = dgm[p_ind,:2]                    
                cnt += 1
    return out


# Obtain the data
img_id = 'mnist'
train_images, train_labels, test_images, test_labels = utils.get_mnist_data()
train_images = train_images.reshape(train_images.shape[0], 28*28).astype('float32')
test_images = test_images.reshape(test_images.shape[0], 28*28).astype('float32')

## Set the params of the filtrations
# Height filtration
num_of_vects = 6
angles = np.linspace(0, math.pi/2, num_of_vects)
dirs = [[round(math.cos(theta),2),round(math.sin(theta),2)] for theta in angles]
dirs = np.array(dirs)

# Radial filtration
center = np.array([[10,10], [10,20], [20,10], [20,20]])
radius = np.array([5, 10, 15])

# Erosion filtration
n_iter_er = np.array([1,2,3,50])

# Dilation filtration
n_iter_dil = np.array([1,3,5,10,50])

params = {'cubical' : None,
         'height': dirs,
         'radial': {'center' : center,
                    'radius' : radius
                    },
         'erosion': n_iter_er,
         'dilation': n_iter_dil
         }
# Get persistence diagrams
pd_train = persistence_diagram.PDiagram(train_images, fil_parms=params, images_id='mnist_train')
pd_test = persistence_diagram.PDiagram(test_images, fil_parms=params, images_id='mnist_test')

# Get train test data
dgms_train = pd_train.get_pds()
dgms_test = pd_test.get_pds()
x_train = pad_diagrams(dgms_train)
x_test = pad_diagrams(dgms_test)
x_train=x_train[:10]
y_train = train_labels
y_test = test_labels

In [ ]:
print(x_train.shape)
import utils 

theta_init = tf.random_normal_initializer()
theta = tf.Variable(name='theta',
                         initial_value=theta_init(shape=(10, 2),dtype=tf.float32))
th0=tf.reshape(theta[0,:],shape=[1,2])
x_o = tf.zeros(shape=(2,))
def _get_tanget_vector(y_theta):
    ''' Maps a single point to the tanget space'''
    y = y_theta[:man_dim]
    theta = y_theta[man_dim:]
    x = utils.Poincare.tf_parametrization(y, theta)
    val = utils.Poincare.tf_log_map_x(x_o, x, 1.)
    if tf.math.is_nan(tf.reduce_sum(val,axis=0)):
        print(y)
    return val
x_train=x_train[:10]
x_train.shape

In [ ]:
max_num_of_pnts = 77
K=10
def __process_dgm(dgm_theta):
    '''
        Process a single diagram, ie for a single homology class
    '''
    
    dgm = dgm_theta[0]
    theta = tf.reshape(dgm_theta[1],shape=[1,man_dim])
    
    tiled_theta = tf.tile(theta, [max_num_of_pnts,1])
    comb = tf.concat([dgm,tiled_theta], axis=1)
    out = tf.map_fn(_get_tanget_vector,comb)
    sums = tf.reduce_sum(out, axis=0)
    x_dgm = utils.Poincare.tf_exp_map_x(x_o, sums, 1.)
    y_dgm = utils.Poincare.tf_chart(x_dgm)
    return (y_dgm,1)

def __process_dgms(dgms):
    print(dgms.shape)
    dgm_0 = dgms[0,:,:] # first homology class
    dgm_1 = dgms[1,:,:] # second one

    # PD of homology class one
    edgm_0 = tf.expand_dims(dgm_0, axis=0)
    tiled_dgm_0 = tf.tile(edgm_0,[K,1,1])
    out_0 = tf.map_fn(__process_dgm, (tiled_dgm_0, theta))[0]

    # PD of homology class two
    edgm_1 = tf.expand_dims(dgm_1, axis=0)
    tiled_dgm_1 = tf.tile(edgm_1,[K,1,1])
    out_1 = tf.map_fn(__process_dgm, (tiled_dgm_1, theta))[0]
    
    return tf.concat([out_0,out_1], axis=0)

def call(inputs):
    '''
        Call method of Keras Layers
    '''
    return tf.map_fn(__process_dgms, inputs)



In [ ]:
from layers import PManifoldLayer, PManifoldModel

In [ ]:
model = PManifoldModel([28,77,3],2,10)

In [ ]:
x_train = np.split(x_train, indices_or_sections= 28, axis=1)


In [ ]:
x_train = [np.squeeze(_) for _ in x_train]
x_train[0].shape

In [ ]:

model.train(x_train,y_train, x_test,y_test)

In [ ]:
%load_ext autoreload
%autoreload 2
batch=np.squeeze(x_train[:10,0,:,:])
batch.shape

In [ ]:
from layers import PManifoldLayer, PManifoldModel
import tensorflow as tf
# pm = PManifoldLayer(10,2,2,77)
# out = pm(batch)
in_layer = []
inputs = []
for _ in range(28):
    pm_layer = PManifoldLayer(10, 2, 2, 77)
    cur_input = tf.keras.Input(shape=(2,77,2))
    inputs.append(cur_input)
    in_layer.append(pm_layer(cur_input))
#model = PManifoldModel([28,77,3],2,10)

In [5]:
import tensorflow as tf
tt=tf.ones(shape=(5,))
pf = tf.pad(tt, [[0,1]])
print(pf)

tf.Tensor([1. 1. 1. 1. 1. 0.], shape=(6,), dtype=float32)
